In [ ]:
import pandas as pd

from SMILESX import main, inference, utils
%load_ext autoreload
%aimport SMILESX
%autoreload 1

%matplotlib inline

#### **Read data file**

In [ ]:
validation_data_dir = "./validation_data/"

In [ ]:
extension = '.csv'

In [ ]:
data_name = 'FreeSolv' # FreeSolv, ESOL, Lipophilicity
prop_tag = ''

In [ ]:
if data_name == 'FreeSolv':
    data_filename = 'FreeSolv_SAMPL'
    prop_tag = 'expt'
elif data_name == 'ESOL':
    data_filename = 'ESOL_delaney-processed'
    prop_tag = 'measured log solubility in mols per litre'
elif data_name == 'Lipophilicity':
    data_filename = 'Lipophilicity'
    prop_tag = 'exp'
else:
    data_filename = data_name
    prop_tag = prop_tag

In [ ]:
sol_data = pd.read_csv(validation_data_dir+data_filename+extension)

In [ ]:
sol_data.head(3)

#### **Observation**
* The column containing the SMILES must be named 'smiles' 

#### **Extract relevant data**

In [ ]:
sol_data = sol_data[['smiles',prop_tag]]

In [ ]:
sol_data.head()

In [ ]:
sol_data.shape

#### **SMILES check from RDKit**

In [ ]:
sol_data, bad_smiles_list = utils.check_smiles(sol_data)

In [ ]:
sol_data.shape

In [ ]:
sol_data.iloc[:,1].values.shape

#### Hyperparameters optimization with GPyOpt (Bayesian optimization)

In [ ]:
dhyp_range = [int(2**itn) for itn in range(3,11)] # 

if data_name != 'Lipophilicity':
    bounds = [
        {'name': 'lstmunits', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'denseunits', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'embedding', 'type': 'discrete', 'domain': dhyp_range}
    ]
else:
    bounds = [
        {'name': 'lstmunits', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'denseunits', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'embedding', 'type': 'discrete', 'domain': dhyp_range}
    ]

In [ ]:
main.Main(data=sol_data, 
          data_name=data_name, 
          data_units='', 
          bayopt_bounds=bounds, 
          k_fold_number = 1, 
          augmentation = True, 
          outdir = "./data/", 
          bayopt_n_rounds = 5, 
          bayopt_on = True, 
          lstmunits_ref = 128, 
          denseunits_ref = 16, 
          embedding_ref = 32, 
          seed_ref = None, 
          n_gpus = 4,
          gpus_list = None, 
          gpus_debug = False,
          batchsize_pergpu = 256,  
          patience = 25, 
          n_epochs = 100)

In [13]:
pred_from_ens = inference.Inference(data_name=data_name, 
                                    smiles_list = ['CC','CCC','C=O','ABC','DEF'], 
                                    data_units = '',
                                    k_fold_number = 3,
                                    augmentation = True, 
                                    outdir = "./data/")

***SMILES_X for inference starts...***


***Checking the SMILES list for inference***

***Data augmentation.***

Enumerated SMILES: 5

***Tokenization of SMILES.***

Full vocabulary: ['pad', 'unk', 'Cl', '[N+]', '\\', ')', '2', '-', 'c', '1', 'P', 'n', '=', ' ', '[C@@]', 'S', '3', 'Br', '#', '/', '[O-]', '[C@@H]', '[C@H]', '[nH]', '5', '4', 'C', 's', '(', 'N', 'F', '[S+2]', 'I', '[C@]', 'O']
Of size: 35

Maximum length of tokenized SMILES: 51 tokens

***Inference of SMILES property done.***


In [14]:
pred_from_ens

,SMILES,ens_pred_mean,ens_pred_sd
0,CC,0.406713,0.0213377
1,CCC,0.43053,0.020964
2,C=O,0.00962523,0.0223681
